In [ ]:
import torch
import torch.optim as optim
import numpy as np
import importlib
import validate_model
import model
import utils
import loss_utils
import model_predict

importlib.reload(model)
importlib.reload(utils)

Y1 = torch.tensor(np.load("data/modality1.npy"), dtype=torch.float32)
Y2 = torch.tensor(np.load("data/modality2.npy"), dtype=torch.float32)

num_demo = Y1.shape[0]
time_len = Y1.shape[1]

X1 = torch.linspace(0, 1, time_len).repeat(num_demo, 1).reshape(num_demo, -1, 1)
X2 = torch.linspace(0, 1, time_len).repeat(num_demo, 1).reshape(num_demo, -1, 1)

In [ ]:
d_x = 1
d_y1 = Y1.shape[2]
d_y2 = Y2.shape[2]
d_N = num_demo
validation_indices = [0]
demo_data = [X1, X2, Y1, Y2]

OBS_MAX = 10 # Hyperparameter: Maximum number of sampled observations per trajectory
batch_size = 8 # Hyperparameter: Batch size for training

In [ ]:

importlib.reload(model)
importlib.reload(utils)
importlib.reload(loss_utils)
importlib.reload(model_predict)
importlib.reload(validate_model)

def train(model, optimizer, EPOCHS):
        
    for i in range(EPOCHS):
    
        obs, mask, x_tar, y1_tar, y2_tar = model.get_training_sample(validation_indices, demo_data, OBS_MAX, 
                                                                           d_N, d_x, d_y1, d_y2, time_len, batch_size)
        optimizer.zero_grad()
        output = model(obs, mask, x_tar)
        loss = model.loss(output, y1_tar, y2_tar, d_y1, d_y2)
        loss.backward()
        optimizer.step()

        if i % 200 == 0:
            losses.append(loss.item())
            error = validate_model.val(model, validation_indices, i, demo_data, d_x, d_y1, d_y2)
            errors.append(error)
            if errors[-1] == min(errors):
                print('Saving model ', i)
                print('loss: ', loss.item())  
                print('mse error: ', error.item())
                torch.save(model.state_dict(), 'save/best_model.pth')

errors = []
losses = []

EPOCHS = 500000

# Generally a good starting point for our model. Please adjust between 1e-3 and 5e-5.
learning_rate = 1e-4

model = model.DualEncoderDecoder(d_x, d_y1, d_y2)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, EPOCHS)